<br>

<div align=center><font color=maroon size=6 style="line-height:40px;"><b>Writing your own callbacks</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TF2 Core: <a href="https://www.tensorflow.org/guide" style="text-decoration:none;">TensorFlow Guide</a> 
    * `TensorFlow > Learn > TensorFlow Core > `Guide > <a href="https://www.tensorflow.org/guide/keras/custom_callback" style="text-decoration:none;">Writing your own callbacks</a>
        * Run in <a href="https://colab.research.google.com/github/tensorflow/docs/blob/snapshot-keras/site/en/guide/keras/custom_callback.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

## Introduction

A callback is a powerful tool to customize the behavior of a Keras model during training, evaluation, or inference. Examples include `tf.keras.callbacks.TensorBoard` to visualize training progress and results with TensorBoard, or `tf.keras.callbacks.ModelCheckpoint` to periodically save your model during training.

In this guide, you will learn what a Keras callback is, what it can do, and how you can build your own. We provide a few demos of simple callback applications to get you started.

<br>

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
print(tf.__version__)

2.8.0


<br>
<br>
<br>

## Keras callbacks overview

<font style="color:maroon;font-size:115%">All callbacks subclass the `keras.callbacks.Callback` class, and override a set of methods called at various stages of training, testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training.

You can pass a list of callbacks (as the keyword argument `callbacks`) to the following model methods:

- `keras.Model.fit()`
- `keras.Model.evaluate()`
- `keras.Model.predict()`
</font>

<br>

## An overview of callback methods

### <font color=maroon>Global methods</font>

* **`on_(train|test|predict)_begin(self, logs=None)`**

  Called at the beginning of `fit`/`evaluate`/`predict`.

<br>

* **`on_(train|test|predict)_end(self, logs=None)`**

  Called at the end of `fit`/`evaluate`/`predict`.

### <font color=maroon>Batch-level methods for training/testing/predicting</font>

* **`on_(train|test|predict)_batch_begin(self, batch, logs=None)`**

  Called right before processing a batch during training/testing/predicting.

<br>

* **`on_(train|test|predict)_batch_end(self, batch, logs=None)`**

  Called at the end of training/testing/predicting a batch. Within this method, `logs` is a dict containing the metrics results.

### <font color=maroon>Epoch-level methods (training only)</font>

* **`on_epoch_begin(self, epoch, logs=None)`**

  Called at the beginning of an epoch during training.

<br>

* **`on_epoch_end(self, epoch, logs=None)`**

  Called at the end of an epoch during training.

<br>
<br>
<br>

## A basic example

Let's take a look at a concrete example. To get started, let's import tensorflow and define a simple Sequential Keras model:

In [3]:
# Define the Keras model to add callbacks to.
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(1, input_dim=784))
    model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
                  loss='mean_squared_error',
                  metrics=['mean_absolute_error'],)

    return model

<br>

Then, load the MNIST data for training and testing from Keras datasets API:

In [4]:
# Load example MNIST data and pre-process it.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

# Limit the data to 1000 samples.
x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

<br>

Now, define a simple custom callback that logs:

- When `fit`/`evaluate`/`predict` starts & ends
- When each epoch starts & ends
- When each training batch starts & ends
- When each evaluation (test) batch starts & ends
- When each inference (prediction) batch starts & ends

In [5]:
class CustomCallback(keras.callbacks.Callback):
    
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))
    
    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))
    
    
    
    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))
    
    
    
    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    
    
    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    
    
    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    
    
    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    
    
    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))


<br>

Let's try it out:

In [6]:
model = get_model()

model.fit(x_train, y_train,
          batch_size=128,
          epochs=1,
          verbose=0,
          validation_split=0.5,
          callbacks=[CustomCallback()],
         )

Starting training; got log keys: []
Start epoch 0 of training; got log keys: []
...Training: start of batch 0; got log keys: []
...Training: end of batch 0; got log keys: ['loss', 'mean_absolute_error']
...Training: start of batch 1; got log keys: []
...Training: end of batch 1; got log keys: ['loss', 'mean_absolute_error']
...Training: start of batch 2; got log keys: []
...Training: end of batch 2; got log keys: ['loss', 'mean_absolute_error']
...Training: start of batch 3; got log keys: []
...Training: end of batch 3; got log keys: ['loss', 'mean_absolute_error']
Start testing; got log keys: []
...Evaluating: start of batch 0; got log keys: []
...Evaluating: end of batch 0; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 1; got log keys: []
...Evaluating: end of batch 1; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 2; got log keys: []
...Evaluating: end of batch 2; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: 

In [7]:
res = model.evaluate(x_test, y_test, 
                     batch_size=128, 
                     verbose=0, 
                     callbacks=[CustomCallback()])

res = model.predict(x_test, batch_size=128, callbacks=[CustomCallback()])

Start testing; got log keys: []
...Evaluating: start of batch 0; got log keys: []
...Evaluating: end of batch 0; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 1; got log keys: []
...Evaluating: end of batch 1; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 2; got log keys: []
...Evaluating: end of batch 2; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 3; got log keys: []
...Evaluating: end of batch 3; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 4; got log keys: []
...Evaluating: end of batch 4; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 5; got log keys: []
...Evaluating: end of batch 5; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 6; got log keys: []
...Evaluating: end of batch 6; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 7; got log keys: []
...Evaluating: end of batch 7

<br>
<br>

### Usage of `logs` dict
<font style="color:maroon;font-size:115%">The `logs` dict contains the loss value, and all the metrics at the end of a batch or epoch.</font> Example includes the loss and mean absolute error.

In [8]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("Up to batch {}, the average loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print("Up to batch {}, the average loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_epoch_end(self, epoch, logs=None):
        print("The average loss for epoch {} is {:7.2f} "
              "and mean absolute error is {:7.2f}.".format(epoch,
                                                           logs["loss"],
                                                           logs["mean_absolute_error"]))


model = get_model()
model.fit(x_train, y_train,
          batch_size=128,
          epochs=2,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback()],)


res = model.evaluate(x_test, y_test,
                     batch_size=128,
                     verbose=0,
                     callbacks=[LossAndErrorPrintingCallback()],)

Up to batch 0, the average loss is   24.16.
Up to batch 1, the average loss is  498.48.
Up to batch 2, the average loss is  340.93.
Up to batch 3, the average loss is  258.51.
Up to batch 4, the average loss is  208.31.
Up to batch 5, the average loss is  174.56.
Up to batch 6, the average loss is  150.46.
Up to batch 7, the average loss is  135.46.
The average loss for epoch 0 is  135.46 and mean absolute error is    6.16.
Up to batch 0, the average loss is    4.42.
Up to batch 1, the average loss is    4.03.
Up to batch 2, the average loss is    4.65.
Up to batch 3, the average loss is    4.62.
Up to batch 4, the average loss is    4.70.
Up to batch 5, the average loss is    4.66.
Up to batch 6, the average loss is    4.55.
Up to batch 7, the average loss is    4.53.
The average loss for epoch 1 is    4.53 and mean absolute error is    1.72.
Up to batch 0, the average loss is    5.20.
Up to batch 1, the average loss is    4.68.
Up to batch 2, the average loss is    4.69.
Up to batch 

<br>
<br>
<br>

## Usage of `self.model` attribute

<font style="color:maroon;font-size:120%">In addition to receiving log information when one of their methods is called, callbacks have access to the model associated with the current round of training/evaluation/inference: `self.model`.
</font>

<font style="color:maroon;font-size:115%">Here are of few of the things you can do with `self.model` in a callback:

<br>

- Set `self.model.stop_training = True` to immediately interrupt training.


- Mutate hyperparameters of the optimizer (available as `self.model.optimizer`), such as `self.model.optimizer.learning_rate`.


- Save the model at period intervals.


- Record the output of `model.predict()` on a few test samples at the end of each epoch, to use as a sanity check during training.


- Extract visualizations of intermediate features at the end of each epoch, to monitor what the model is learning over time.


- etc.</font>

Let's see this in action in a couple of examples.

<br>
<br>
<br>

## <font color=maroon>**Examples of Keras callback applications**</font>

### Early stopping at minimum loss

<font style="color:maroon;font-size:115%">This first example shows the creation of a `Callback` that stops training when the minimum of loss has been reached, by setting the attribute `self.model.stop_training` (boolean). Optionally, you can provide an argument `patience` to specify how many epochs we should wait before stopping after having reached a local minimum.

`tf.keras.callbacks.EarlyStopping` provides a more complete and general implementation.</font>

In [9]:
import numpy as np

In [10]:
class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

    Arguments:
        patience: Number of epochs to wait after min has been hit. After this number
                  of no improvement, training stops.
    """
    
    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None
    
    
    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as Infinity.
        self.best = np.Inf
    
    
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get('loss')
        
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)
    
    
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

In [11]:
model = get_model()
model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=30,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()],)

Up to batch 0, the average loss is   31.42.
Up to batch 1, the average loss is  377.33.
Up to batch 2, the average loss is  258.98.
Up to batch 3, the average loss is  196.80.
Up to batch 4, the average loss is  159.05.
The average loss for epoch 0 is  159.05 and mean absolute error is    7.88.
Up to batch 0, the average loss is    7.81.
Up to batch 1, the average loss is    7.08.
Up to batch 2, the average loss is    6.43.
Up to batch 3, the average loss is    6.28.
Up to batch 4, the average loss is    6.35.
The average loss for epoch 1 is    6.35 and mean absolute error is    2.06.
Up to batch 0, the average loss is    5.37.
Up to batch 1, the average loss is    6.24.
Up to batch 2, the average loss is    5.70.
Up to batch 3, the average loss is    6.45.
Up to batch 4, the average loss is    8.61.
The average loss for epoch 2 is    8.61 and mean absolute error is    2.33.
Restoring model weights from the end of the best epoch.
Epoch 00003: early stopping


<br>
<br>

### Learning rate scheduling

<font style="color:maroon;font-size:115%">In this example, we show how a custom Callback can be used to dynamically change the learning rate of the optimizer during the course of training.

See `callbacks.LearningRateScheduler` for a more general implementations.</font>

In [12]:
class CustomLearningRateScheduler(keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

    Arguments:
        schedule: a function that takes an epoch index (integer, indexed from 0) 
                  and current learning rate as inputs 
                  and returns a new learning rate as output (float).
    """

    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        
        # Set the value back to the optimizer before this epoch starts
        
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))


LR_SCHEDULE = [# (epoch to start, learning rate) tuples
               (3, 0.05),
               (6, 0.01),
               (9, 0.005),
               (12, 0.001),]


def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr


model = get_model()
model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=15,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(),
                     CustomLearningRateScheduler(lr_schedule),],
         )


Epoch 00000: Learning rate is 0.1000.
Up to batch 0, the average loss is   28.94.
Up to batch 1, the average loss is  446.10.
Up to batch 2, the average loss is  306.62.
Up to batch 3, the average loss is  232.15.
Up to batch 4, the average loss is  187.20.
The average loss for epoch 0 is  187.20 and mean absolute error is    8.17.

Epoch 00001: Learning rate is 0.1000.
Up to batch 0, the average loss is    8.70.
Up to batch 1, the average loss is    7.72.
Up to batch 2, the average loss is    6.80.
Up to batch 3, the average loss is    6.22.
Up to batch 4, the average loss is    5.80.
The average loss for epoch 1 is    5.80 and mean absolute error is    1.94.

Epoch 00002: Learning rate is 0.1000.
Up to batch 0, the average loss is    5.56.
Up to batch 1, the average loss is    6.21.
Up to batch 2, the average loss is    6.02.
Up to batch 3, the average loss is    6.25.
Up to batch 4, the average loss is    7.12.
The average loss for epoch 2 is    7.12 and mean absolute error is    2

<br>
<br>

### Built-in Keras callbacks
<font style="color:maroon;font-size:115%">Be sure to check out the existing Keras callbacks by reading the [API docs](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/).<br>

Applications include logging to CSV, saving the model, visualizing metrics in TensorBoard, and a lot more!</font>

<br>
<br>
<br>